In [13]:
print('hello world')

Base.Meta.ParseError: ParseError:
# Error @ /Users/julkakubisa/Documents/PRACA MAGISTERSKA/spaced_repetition_algorithms/notebooks/duolingo_EDA.ipynb:1:8
print('hello world')
#      └─────────┘ ── character literal contains multiple characters

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy.stats as stats
import seaborn as sns

%matplotlib inline


ArgumentError: ArgumentError: Package pandas not found in current path.
- Run `import Pkg; Pkg.add("pandas")` to install the pandas package.

In [7]:
df = pd.read_csv("/Users/julkakubisa/Documents/PRACA MAGISTERSKA/spaced_repetition_algorithms/data/13 million Duolingo student learning traces.csv")

# Take a sample of dataframe 
df = df.sample(500000)

UndefVarError: UndefVarError: `pd` not defined

In [ ]:
""" 
p_recall - session_correct/session_seen
timestamp - UNIX timestamp of the current lesson
delta - time (in seconds) since the last lesson that included this word
user_id - student user ID
learning_language - language being learned
ui_language - user interface language (presumably native to the student)
lexeme_id - system ID for the lexeme tag (i.e., word)
lexeme_string - lexeme tag for word
history_seen - total times user has seen the word prior to this lesson
history_correct - total times user has been correct for the word prior to this lesson
session_seen - times the user saw the word during this lesson
session_correct - times the user got the word correct during this lesson
"""


In [ ]:
df.reset_index(inplace=True, drop=True)
df.drop('timestamp', axis=1, inplace=True)
df

In [ ]:
# Check the types
df.dtypes

In [ ]:
# Check for na
df.isna().any()

In [ ]:
# Check for duplicates
df.duplicated().any()

In [ ]:
# Create new column ui language - learning language 
df['lang'] = df['learning_language'] + '-' + df['ui_language']

In [ ]:
# Get numeric columns 
df_numeric = df.select_dtypes(np.number)
features = df_numeric.columns.tolist()

# Get object columns  
df_obj = df.select_dtypes(include='object')

In [ ]:
"""
p_recall (proportion of correct answers to total answers in one session): the mean is 0.9, the median is 1. More than 75% users can answer all questions correctly in one session. 
delta: the mean is 3.36 days between sessions, the median is 0.35 day. Max value is 466 days, 75% users have less than 5.12 days between sessions. Likely outliers. 
history_seen: median is 6 times seen prior to this session, the max is very high (13 518)
history_correct: mean is 6 times correct, standard deviation is very high. 
session_seen: usually around 1 time seen during session, mean 1.8. 
session_correct: mean 1.6, median 1.0 
"""

In [ ]:
# Describe numerical columns 
df.describe().map(lambda x: f"{x:0.3f}")

In [ ]:
# Describe by language
for column in features:
    print('Stats for:', column)
    print(df.groupby('learning_language')[column].describe())

In [ ]:
# Describe categorical columns
df.describe(include=['O'])

In [ ]:
print('a')

In [ ]:
# stats.zscore(df_numeric)

In [ ]:
# Numeric analysis -> outliers
stats.zscore(df_numeric)
threshold_z = 3
z = np.abs(stats.zscore(df_numeric))       
mask = np.abs(z) < threshold_z
df = df[mask] 
# df = df[(z < threshold_z).all(axis=1)]
print("Original DataFrame Shape:", df_numeric.shape[0])
print("DataFrame Shape after Removing Outliers:", df.shape[0])
print('Rows removed', df_numeric.shape[0]- df.shape[0])


# Delete also individually for columns
# outlier_indices = np.where(z > threshold_z)[0]
# no_outliers = df_diabetics.drop(outlier_indices)



In [ ]:
mask

In [ ]:
# langs_sizes = df.groupby(["ui_language", "learning_language"]).size().to_frame('count')
# langs_sizes['% obs'] = (langs_sizes['count']/langs_sizes['count'].sum())*100
# langs_sizes

langs_sizes = df.groupby(['lang']).size().to_frame('count')
langs_sizes['% obs'] = (langs_sizes['count']/langs_sizes['count'].sum())*100
langs_sizes

target_langs = df.groupby(['learning_language']).size().to_frame('count')
target_langs['% obs'] = (target_langs['count']/target_langs['count'].sum())*100
target_langs

In [ ]:
sessions_size = df.groupby(["session_correct", "session_seen"]).size().to_frame('count')
sessions_size

In [ ]:
# For ints and objects! 
_, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 4))
sns.countplot(x='session_seen', data=df, ax=axes[0,0])
sns.countplot(x='session_correct', data=df, ax=axes[0,1])
sns.countplot(x='learning_language', data=df, ax=axes[1,0])
sns.countplot(x='ui_language', data=df, ax=axes[1,1])
plt.tight_layout()
plt.show()

# sns.countplot(x='lang', data=df)


In [ ]:
features.remove('timestamp')

In [ ]:
features

In [ ]:
# Boxplots 

fig, axs = plt.subplots(2, 2, figsize=(12, 8))

# Plot each boxplot on the correct subplot
axs[0,0].boxplot(df['session_seen'])
axs[0, 0].set_title('Session Seen')

axs[0,1].boxplot(df['history_correct'])
axs[0, 1].set_title('history_correct')

axs[1,0].boxplot(df['p_recall'])
axs[1, 0].set_title('P Recall')

axs[1,1].boxplot(df['delta'])
axs[1, 1].set_title('Delta')

fig.suptitle('Shared boxplots')
fig.tight_layout()
fig.subplots_adjust(top=0.9)  # Adjust the top to make room for the super title

plt.show()

In [ ]:
# Boxplots one by one
for y in features:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x="learning_language", y=y, data=df)
    plt.title(f'Boxplot of {y} by Learning Language')
    plt.show()


In [ ]:
# Histograms
df[features].hist(figsize=(10,8), color='green')
plt.ticklabel_format(style = 'plain')

In [ ]:
df[features].plot(kind='density',subplots=True,layout=(2,2),
                  sharex=False,figsize=(10,4))
plt.show()

In [ ]:
# Relationship -> delta with things 

fig, axs = plt.subplots(2, 2, figsize=(12, 8))
axs[0,0].scatter(df_500['delta'], df_500['history_correct'])
axs[1,0].scatter(df_500['p_recall'], df_500['history_correct'])
axs[0,1].scatter(df_500['session_correct'], df_500['delta'])
axs[1,1].scatter(df_500['p_recall'], df_500['delta'])

plt.show()


# Divide 
# for cat in categories:
#     dataset[cat] = dataset.genres.apply(lambda s: int(cat in s))
# dataset.head()


    # plt.scatter(df['Total day minutes'], df['Total night minutes'])
    # corr_matrix = df[numerical].corr()
    # sns.heatmap(corr_matrix)
    
    # sns.catplot(x='Churn', y='Total day minutes',col='Customer service calls',
    #         data=df[df['Customer service calls'] < 8], kind="box",
    #         col_wrap=4, height=3,aspect=.8)